In [1]:
pwd

'/Users/robincole/Documents/Github/HASS-folder-watcher/development'

In [17]:
import datetime
import os
import glob

In [218]:
path = '/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/'
os.path.isdir(path)

True

In [221]:
glob.glob(path + '**/' + '*', recursive = True)

['/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/file2.txt',
 '/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/second_dummy',
 '/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/file1.txt',
 '/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/second_dummy/file3.txt']

In [40]:
def get_timestamp(file_path):
    """Return the timestamp of file."""
    mtime = os.stat(file_path).st_mtime
    return datetime.datetime.fromtimestamp(mtime).isoformat()

In [88]:
def get_files_dict(folder_path, filter_term):
    """Return the dict of file paths and mod times, applying filter."""
    query = folder_path + filter_term
    files_list = glob.glob(query)
    files_list = [f for f in files_list if os.path.isfile(f)]
    files_dict = {f:get_timestamp(f) for f in files_list}
    return files_dict

In [206]:
def get_size(files_list):
    """Return the sum of the size in bytes of files in the list."""
    size_list = [os.stat(f).st_size for f in files_list]
    return sum(size_list)

We start with an emptry dictionary, which we populate from the filtered list of path files

In [120]:
files_record = {}

Now iterate current_files, do comparisons to files_record

In [213]:
current_files = get_files_dict(path, '*')

for file_path in set(list(current_files.keys()) + list(files_record.keys())):            
    # Logic to process files
    if file_path not in files_record:
        print("{} added".format(file_path))
        files_record[file_path] = current_files[file_path] # Add the entry
        
    elif file_path not in current_files: # If deleted
        print("{} deleted".format(file_path))
        files_record.pop(file_path, None)
        
    elif file_path in files_record and current_files:
        if files_record[file_path] != current_files[file_path]: # If in both registers and modified
            print("{} modified".format(file_path))
            files_record[file_path] = current_files[file_path] # Reassign

/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/file2.txt modified


In [214]:
current_files

{'/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/file1.txt': '2018-02-28T09:05:14.348702',
 '/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/file2.txt': '2018-03-03T14:51:07.442320'}

In [215]:
files_record

{'/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/file1.txt': '2018-02-28T09:05:14.348702',
 '/Users/robincole/Documents/Github/HASS-folder-watcher/development/dummy_folder/file2.txt': '2018-03-03T14:51:07.442320'}

In [212]:
len(files_record)

2

In [207]:
get_size(list(files_record.keys()))

30

## Class implementation

In [15]:
"""The dict self._files is updated and events published on changes to the dict."""

class Watcher():
    def __init__(self, path, hass):
        self._path = path
        self.hass = hass
        self._updated = False
        self._files = get_files(self._path)

    def update(self):
        """Update the watcher."""
        current_files = []
        previous_files = list(self._files.keys())
        
        with os.scandir(self._path) as folder:
            for entry in folder:
                if not entry.name.startswith('.') and entry.is_file(): 
                    mtime = get_timestamp(entry)
                    fname = entry.name
                    current_files.append(fname)  # Keep record of current files   
            
                    if not fname in self._files:
                        print("{} added".format(fname))
                        # self.hass.bus.fire('file_added', { 'file': fname })
                        self._files[fname] = mtime # Add the entry
                
                    elif fname in self._files and self._files[fname] != mtime: # If exists and modified
                        print("{} modified".format(fname))
                        # self.hass.bus.fire('file_modified', { 'file': fname })
                        self._files[fname] = mtime # Update timestamp

            # Check if any files deleted                
            for fname in list(set(previous_files) - set(current_files)):
                print("{} deleted".format(fname))
                # self.hass.bus.fire('file_deleted', { 'file': fname })
                self._files.pop(fname, None) # Delete the entry

In [16]:
hass = 'hass'
my_watcher = Watcher(path, hass)

In [17]:
my_watcher.update()

In [10]:
my_watcher.update()

file2.txt modified


In [23]:
my_watcher.update()

file2 copy.txt added


In [24]:
my_watcher.update()

file2.txt modified
file2 copy.txt deleted


In [34]:
path = '/Users/robincole/.homeassistant/images/'

In [36]:
os.path.split(os.path.split(path)[0])[1]

'images'

In [27]:
pwd

'/Users/robincole/Documents/Github/HASS-folder-watcher/development'